In [1]:
dataName = 'crudeOilFeb'
dataFrequency='5m'

import os
import sys  
module_path = os.path.abspath('../../'+os.path.join('.'))

sys.path.append(module_path)

print("module_path >>> " + module_path + " <<< appended to sys path - Mandatory for the pythons files in src folder to be recognized")



module_path >>> /Users/493055/Desktop/dev/erai-v0.3/src <<< appended to sys path - Mandatory for the pythons files in src folder to be recognized


In [3]:
# from dataPreprocessing import *

# from dataPreprocessing.preProcessData import collateData

# df = collateData(dataName,dataFrequency)



In [6]:
import os
import sys
import traceback

import pandas as pd
import glob

from utils.errorHandler import handleError
from utils.common import getProjectRootFolderPath

from utils.fileFolderManipulations import getParentFolder
from utils.fileFolderManipulations import createFolder

from config.config import getAppConfigData
from config.config import setAppConfigData

from fastai.tabular import add_datepart

from dataPreparation.simpleFeatures import getFollowingHolidaysDaysStamp
from dataPreparation.simpleFeatures import getPriorHoliDaysStamps

print(' data pre-processing >> imported dependencies')

relativeDataFolderPath = 'data/'+dataName+'/raw/' + dataFrequency

# Variable to hold the original source folder path which is calculated from the input relative path of the source folder (relativeDataFolderPath)
# using various python commands like os.path.abspath and os.path.join
projectRootFolderPath = None

# Variable to hold a dataframe created with the data from input data files in the relativeDataFolderPath provided
inputRawDataDF = None

# Variable to hold the original source folder path which is calculated from the input relative path of the source folder (relativeDataFolderPath)
# using various python commands like os.path.abspath and os.path.join
dataFolderPath = None

# Variable to hold query like value of python to query all json file names in the source folder (dataFolderPath).
# Will be used in the glob function to execute the query
json_pattern = None

# Variable to contain the list of all input json file names in the source folder (dataFolderPath)
file_list = None

# return values of this method
# -------------------------------------------------------------------------------
# Current methods return value initialized to false. Will be maked as true
# after every single line in the method has been executed with out errors
returnValue = False
# complete filepath of the csv file with the processed raw data
outputFilePath = None
outputFolderName = None


 data pre-processing >> imported dependencies


In [7]:
# caluclate the deployment directory path of the current juypter node in the operating system
projectRootFolderPath = getProjectRootFolderPath()

# TO BE MODIFIED - NOT SURE WHY I USED THIS - WILL HAVE TO CHECK
pd.set_option('display.max_columns', None)

# creating pandas dataframe references for further modification
inputRawDataDF = pd.DataFrame()

# calculating the complete data folder path of the relative path provided as parameter
dataFolderPath = projectRootFolderPath + '/'+relativeDataFolderPath

# creating OS queryable object for python to work with to find json files in the dataFolderPath calcuated in the previous step
json_pattern = os.path.join(dataFolderPath, '*.json')

# store all the json file paths in the dataFolderPath for further processing
file_list = glob.glob(json_pattern)

In [8]:
file_list

['/Users/493055/Desktop/dev/erai-v0.3/data/crudeOilFeb/raw/5m/01.json',
 '/Users/493055/Desktop/dev/erai-v0.3/data/crudeOilFeb/raw/5m/06.json',
 '/Users/493055/Desktop/dev/erai-v0.3/data/crudeOilFeb/raw/5m/07.json',
 '/Users/493055/Desktop/dev/erai-v0.3/data/crudeOilFeb/raw/5m/04.json',
 '/Users/493055/Desktop/dev/erai-v0.3/data/crudeOilFeb/raw/5m/08.json',
 '/Users/493055/Desktop/dev/erai-v0.3/data/crudeOilFeb/raw/5m/09.json',
 '/Users/493055/Desktop/dev/erai-v0.3/data/crudeOilFeb/raw/5m/05.json',
 '/Users/493055/Desktop/dev/erai-v0.3/data/crudeOilFeb/raw/5m/02.json',
 '/Users/493055/Desktop/dev/erai-v0.3/data/crudeOilFeb/raw/5m/03.json']

In [10]:
file_list.sort()
file_list


['/Users/493055/Desktop/dev/erai-v0.3/data/crudeOilFeb/raw/5m/01.json',
 '/Users/493055/Desktop/dev/erai-v0.3/data/crudeOilFeb/raw/5m/02.json',
 '/Users/493055/Desktop/dev/erai-v0.3/data/crudeOilFeb/raw/5m/03.json',
 '/Users/493055/Desktop/dev/erai-v0.3/data/crudeOilFeb/raw/5m/04.json',
 '/Users/493055/Desktop/dev/erai-v0.3/data/crudeOilFeb/raw/5m/05.json',
 '/Users/493055/Desktop/dev/erai-v0.3/data/crudeOilFeb/raw/5m/06.json',
 '/Users/493055/Desktop/dev/erai-v0.3/data/crudeOilFeb/raw/5m/07.json',
 '/Users/493055/Desktop/dev/erai-v0.3/data/crudeOilFeb/raw/5m/08.json',
 '/Users/493055/Desktop/dev/erai-v0.3/data/crudeOilFeb/raw/5m/09.json']

In [11]:
for file in file_list:
    print("reading input file >>> " + file + " ...")
    data=None
    try:
        # read json data from unformatted json file
        data = pd.read_json(file, lines=True)

        if isinstance(data, str):
            data = data['data'][0]['candles']
        elif isinstance(data,pd.DataFrame):
            data = data['data'][0]['candles']
        else:
            data = data.values[0][0]['candles']

    except ValueError:
        # read data from formatted / json linted json file
        data = pd.read_json(file)

        data = data['data'][0]


    inputRawDataDF = inputRawDataDF.append(data, ignore_index=True)
    print("File read - SUCCESS")

#assign column names
inputRawDataDF.columns = ['date-time', 'open',
                          'high', 'low', 'close', 'quantity', 'dont-know']

reading input file >>> /Users/493055/Desktop/dev/erai-v0.3/data/crudeOilFeb/raw/5m/01.json ...
File read - SUCCESS
reading input file >>> /Users/493055/Desktop/dev/erai-v0.3/data/crudeOilFeb/raw/5m/02.json ...
File read - SUCCESS
reading input file >>> /Users/493055/Desktop/dev/erai-v0.3/data/crudeOilFeb/raw/5m/03.json ...
File read - SUCCESS
reading input file >>> /Users/493055/Desktop/dev/erai-v0.3/data/crudeOilFeb/raw/5m/04.json ...
File read - SUCCESS
reading input file >>> /Users/493055/Desktop/dev/erai-v0.3/data/crudeOilFeb/raw/5m/05.json ...
File read - SUCCESS
reading input file >>> /Users/493055/Desktop/dev/erai-v0.3/data/crudeOilFeb/raw/5m/06.json ...
File read - SUCCESS
reading input file >>> /Users/493055/Desktop/dev/erai-v0.3/data/crudeOilFeb/raw/5m/07.json ...
File read - SUCCESS
reading input file >>> /Users/493055/Desktop/dev/erai-v0.3/data/crudeOilFeb/raw/5m/08.json ...
File read - SUCCESS
reading input file >>> /Users/493055/Desktop/dev/erai-v0.3/data/crudeOilFeb/raw/

In [21]:
rawdataDf  = inputRawDataDF 
rawdataDf.index = rawdataDf['date-time']
rawdataDf = rawdataDf.sort_index()

In [23]:
rawdataDf.shape

(8859, 7)

In [24]:
rawdataDf=rawdataDf.groupby(inputRawDataDF.index).last()
rawdataDf.shape

(8262, 7)

In [25]:
rawdataDf.to_csv(projectRootFolderPath+'/01.crudeoil_feb_2020.csv')

In [27]:
dataName = 'crudeoil_india'

import os
import sys
import traceback

import pandas as pd
import glob

from utils.errorHandler import handleError
from utils.common import getProjectRootFolderPath

from utils.fileFolderManipulations import getParentFolder
from utils.fileFolderManipulations import createFolder

from config.config import getAppConfigData
from config.config import setAppConfigData

from fastai.tabular import add_datepart

from dataPreparation.simpleFeatures import getFollowingHolidaysDaysStamp
from dataPreparation.simpleFeatures import getPriorHoliDaysStamps

print(' data pre-processing >> imported dependencies')

relativeDataFolderPath = 'data/'+dataName+'/raw/' + dataFrequency

# Variable to hold the original source folder path which is calculated from the input relative path of the source folder (relativeDataFolderPath)
# using various python commands like os.path.abspath and os.path.join
projectRootFolderPath = None

# Variable to hold a dataframe created with the data from input data files in the relativeDataFolderPath provided
inputRawDataDF = None

# Variable to hold the original source folder path which is calculated from the input relative path of the source folder (relativeDataFolderPath)
# using various python commands like os.path.abspath and os.path.join
dataFolderPath = None

# Variable to hold query like value of python to query all json file names in the source folder (dataFolderPath).
# Will be used in the glob function to execute the query
json_pattern = None

# Variable to contain the list of all input json file names in the source folder (dataFolderPath)
file_list = None

# return values of this method
# -------------------------------------------------------------------------------
# Current methods return value initialized to false. Will be maked as true
# after every single line in the method has been executed with out errors
returnValue = False
# complete filepath of the csv file with the processed raw data
outputFilePath = None
outputFolderName = None

# caluclate the deployment directory path of the current juypter node in the operating system
projectRootFolderPath = getProjectRootFolderPath()

# TO BE MODIFIED - NOT SURE WHY I USED THIS - WILL HAVE TO CHECK
pd.set_option('display.max_columns', None)

# creating pandas dataframe references for further modification
inputRawDataDF = pd.DataFrame()

# calculating the complete data folder path of the relative path provided as parameter
dataFolderPath = projectRootFolderPath + '/'+relativeDataFolderPath

# creating OS queryable object for python to work with to find json files in the dataFolderPath calcuated in the previous step
json_pattern = os.path.join(dataFolderPath, '*.json')

# store all the json file paths in the dataFolderPath for further processing
file_list = glob.glob(json_pattern)

file_list

 data pre-processing >> imported dependencies


['/Users/493055/Desktop/dev/erai-v0.3/data/crudeoil_india/raw/5m/01.json',
 '/Users/493055/Desktop/dev/erai-v0.3/data/crudeoil_india/raw/5m/02.json',
 '/Users/493055/Desktop/dev/erai-v0.3/data/crudeoil_india/raw/5m/03.json']

In [28]:
file_list.sort()
for file in file_list:
    print("reading input file >>> " + file + " ...")
    data=None
    try:
        # read json data from unformatted json file
        data = pd.read_json(file, lines=True)

        if isinstance(data, str):
            data = data['data'][0]['candles']
        elif isinstance(data,pd.DataFrame):
            data = data['data'][0]['candles']
        else:
            data = data.values[0][0]['candles']

    except ValueError:
        # read data from formatted / json linted json file
        data = pd.read_json(file)

        data = data['data'][0]


    inputRawDataDF = inputRawDataDF.append(data, ignore_index=True)
    print("File read - SUCCESS")

#assign column names
inputRawDataDF.columns = ['date-time', 'open',
                          'high', 'low', 'close', 'quantity', 'dont-know']

rawdataDf  = inputRawDataDF 
rawdataDf.index = rawdataDf['date-time']
rawdataDf = rawdataDf.sort_index()

reading input file >>> /Users/493055/Desktop/dev/erai-v0.3/data/crudeoil_india/raw/5m/01.json ...
File read - SUCCESS
reading input file >>> /Users/493055/Desktop/dev/erai-v0.3/data/crudeoil_india/raw/5m/02.json ...
File read - SUCCESS
reading input file >>> /Users/493055/Desktop/dev/erai-v0.3/data/crudeoil_india/raw/5m/03.json ...
File read - SUCCESS


In [29]:
rawdataDf.shape

(6963, 7)

In [30]:
rawdataDf=rawdataDf.groupby(inputRawDataDF.index).last()
rawdataDf.shape

(6963, 7)

In [31]:
rawdataDf.to_csv(projectRootFolderPath+'/01.crudeoil_mar_2020.csv')